# Least-squares fit with multivariate model

This is a tutorial on how to fit a multivariate function z = f(x, y) to data with the builtin LeastSquares cost function.

We fit a scaled multivariate normal distribution.

In [ ]:
from iminuit import Minuit
import numpy as np
from scipy.stats import multivariate_normal as norm
from iminuit.cost import LeastSquares
from matplotlib import pyplot as plt

# docs of LeastSquares class explain how to set up model function
def model(x_y, n, mux, muy, sigmax, sigmay, rho):
    C = np.empty((2, 2))
    C[0,0] = sigmax ** 2
    C[1,1] = sigmay ** 2
    C[0,1] = C[1, 0] = rho * sigmax * sigmay
    return n * norm.pdf(np.transpose(x_y), (mux, muy), C)

# generate a regular grid in x and y
x = np.linspace(-1, 1, 10)
y = np.linspace(-1, 1, 10)
X, Y = np.meshgrid(x, y)
x = X.flatten()
y = Y.flatten()

# model truth
Z = model((x, y), 10, 0, 0, 1, 0.5, 0.1)

# add some noise
rng = np.random.default_rng(1)
Zerr = 0.1 * Z
Z += rng.normal(0, Zerr)

plt.scatter(x, y, c=Z)

In [ ]:
cost = LeastSquares((x, y), Z, Zerr, model)

m = Minuit(cost, 1, 0, 0, 1, 1, 0.0)
m.limits["n", "sigmax", "sigmay"] = (0, None)
m.limits["rho"] = (-1, 1)
m.migrad()

Multivarate fits are difficult to check by eye. Here we use color to indicate the function value.

To guarantee that plot of the function and the plot of the data use the same color scale, we use the same normalising function for pyplot.pcolormesh and pyplot.scatter.

In [ ]:
xm = np.linspace(-1, 1, 100)
ym = np.linspace(-1, 1, 100)
Xm, Ym = np.meshgrid(xm, ym)
xm = Xm.flatten()
ym = Ym.flatten()

qm = plt.pcolormesh(Xm, Ym, model((xm, ym), *m.values).reshape(Xm.shape))
plt.scatter(x, y, c=Z, edgecolors="w", norm=qm.norm)
plt.colorbar()
plt.clim(0, None)